# Agentic Website Analysis Demo
## Multi-Agent System for Marketing Team Autonomy

This demo showcases a working multi-agent system that analyzes websites and provides actionable design recommendations for Marketing teams.

### Demo Flow:
1. **Web Acquisition Agent** - Fetches and analyzes website structure
2. **Analysis Agent** - AI-powered analysis of design patterns and UX issues  
3. **Design Suggestion Agent** - Generates Marketing-focused improvement recommendations

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json
from dotenv import load_dotenv
from anthropic import Anthropic
import re
from datetime import datetime

# Load environment variables
load_dotenv()

# Initialize Anthropic client
try:
    client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
except Exception as e:
    print(f"⚠️  Anthropic client initialization failed: {e}")
    client = None

print("✅ Environment setup complete!")
print(f"📅 Demo timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Environment setup complete!
📅 Demo timestamp: 2025-07-02 18:36:57


In [2]:
## 🤖 Agent 1: Web Acquisition Agent

class WebAcquisitionAgent:
    """Responsible for fetching and analyzing website structure"""
    
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        })
    
    def fetch_website(self, url):
        """Fetch website content and extract key elements"""
        print(f"🔍 Web Acquisition Agent: Analyzing {url}")
        
        try:
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract key website elements
            analysis = {
                'url': url,
                'title': soup.find('title').get_text(strip=True) if soup.find('title') else 'No title',
                'meta_description': self._get_meta_description(soup),
                'headings': self._extract_headings(soup),
                'navigation': self._extract_navigation(soup),
                'images': self._extract_images(soup, url),
                'calls_to_action': self._extract_ctas(soup),
                'content_sections': self._extract_content_sections(soup),
                'forms': self._extract_forms(soup)
            }
            
            print("✅ Website acquisition complete!")
            return analysis
            
        except Exception as e:
            print(f"❌ Error fetching website: {str(e)}")
            return None
    
    def _get_meta_description(self, soup):
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        return meta_desc.get('content', '') if meta_desc else ''
    
    def _extract_headings(self, soup):
        headings = {}
        for i in range(1, 7):
            h_tags = soup.find_all(f'h{i}')
            if h_tags:
                headings[f'h{i}'] = [h.get_text(strip=True) for h in h_tags[:5]]  # Limit to first 5
        return headings
    
    def _extract_navigation(self, soup):
        nav_elements = soup.find_all(['nav', 'ul', 'ol'])
        nav_items = []
        for nav in nav_elements[:3]:  # Limit to first 3 nav elements
            links = nav.find_all('a')
            nav_items.extend([link.get_text(strip=True) for link in links[:10]])  # Limit links
        return nav_items[:15]  # Total limit
    
    def _extract_images(self, soup, base_url):
        imgs = soup.find_all('img')
        images = []
        for img in imgs[:10]:  # Limit to first 10 images
            src = img.get('src', '')
            alt = img.get('alt', '')
            if src:
                full_url = urljoin(base_url, src)
                images.append({'src': full_url, 'alt': alt})
        return images
    
    def _extract_ctas(self, soup):
        # Look for common CTA patterns
        cta_selectors = ['button', 'a[class*="btn"]', 'a[class*="cta"]', 'input[type="submit"]']
        ctas = []
        for selector in cta_selectors:
            elements = soup.select(selector)
            for elem in elements[:5]:  # Limit per selector
                text = elem.get_text(strip=True)
                if text and len(text) < 50:  # Reasonable CTA length
                    ctas.append(text)
        return list(set(ctas))[:10]  # Remove duplicates, limit total
    
    def _extract_content_sections(self, soup):
        # Look for main content areas
        sections = []
        for tag in ['main', 'section', 'article', 'div[class*="content"]']:
            elements = soup.select(tag)
            for elem in elements[:3]:  # Limit sections
                text = elem.get_text(strip=True)
                if len(text) > 100:  # Substantial content
                    sections.append(text[:200] + '...' if len(text) > 200 else text)
        return sections
    
    def _extract_forms(self, soup):
        forms = soup.find_all('form')
        form_data = []
        for form in forms[:3]:  # Limit forms
            inputs = form.find_all(['input', 'select', 'textarea'])
            form_info = {
                'action': form.get('action', ''),
                'method': form.get('method', 'GET'),
                'fields': [inp.get('name', inp.get('type', 'unknown')) for inp in inputs[:5]]
            }
            form_data.append(form_info)
        return form_data

# Initialize the Web Acquisition Agent
web_agent = WebAcquisitionAgent()
print("🤖 Web Acquisition Agent initialized!")

🤖 Web Acquisition Agent initialized!


In [3]:
## 🧠 Agent 2: Analysis Agent

class AnalysisAgent:
    """AI-powered analysis of website design patterns and UX issues"""
    
    def __init__(self, anthropic_client):
        self.client = anthropic_client
    
    def analyze_website(self, website_data):
        """Analyze website structure and identify design patterns and issues"""
        print("🧠 Analysis Agent: Processing website data...")
        
        # Prepare structured data for AI analysis
        analysis_prompt = f"""
        You are a senior UX/UI analyst reviewing a website for a Marketing team. 
        Analyze the following website data and provide insights on:
        1. Overall design patterns and user experience
        2. Marketing effectiveness (messaging, CTAs, user journey)
        3. Accessibility and usability concerns
        4. Content hierarchy and information architecture
        
        Website Data:
        Title: {website_data['title']}
        Meta Description: {website_data['meta_description']}
        
        Headings Structure:
        {json.dumps(website_data['headings'], indent=2)}
        
        Navigation Items: {website_data['navigation'][:10]}
        
        Call-to-Action Buttons: {website_data['calls_to_action']}
        
        Forms Present: {len(website_data['forms'])} forms detected
        
        Images: {len(website_data['images'])} images found
        
        Content Sections: {len(website_data['content_sections'])} main content areas
        
        Provide a structured analysis in the following format:
        
        DESIGN PATTERNS:
        - [Key design patterns observed]
        
        MARKETING EFFECTIVENESS:
        - [Assessment of messaging and conversion elements]
        
        USER EXPERIENCE ISSUES:
        - [Potential usability problems]
        
        ACCESSIBILITY CONCERNS:
        - [Basic accessibility observations]
        
        INFORMATION ARCHITECTURE:
        - [Content organization and hierarchy assessment]
        
        Keep your analysis concise but actionable for a Marketing team.
        """
        
        try:
            response = self.client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=1000,
                temperature=0.3,
                messages=[
                    {"role": "user", "content": analysis_prompt}
                ]
            )
            
            analysis_result = response.content[0].text
            print("✅ Website analysis complete!")
            return analysis_result
            
        except Exception as e:
            print(f"❌ Error in analysis: {str(e)}")
            return f"Analysis failed: {str(e)}"

# Initialize the Analysis Agent  
analysis_agent = AnalysisAgent(client)
print("🧠 Analysis Agent initialized!")

🧠 Analysis Agent initialized!


In [4]:
## 💡 Agent 3: Design Suggestion Agent

class DesignSuggestionAgent:
    """Generates Marketing-focused improvement recommendations"""
    
    def __init__(self, anthropic_client):
        self.client = anthropic_client
    
    def generate_suggestions(self, website_data, analysis_result):
        """Generate specific, actionable design improvements for Marketing team"""
        print("💡 Design Suggestion Agent: Creating recommendations...")
        
        suggestion_prompt = f"""
        You are a Marketing-focused design consultant. Based on the website analysis below, 
        generate 3-5 specific, actionable recommendations that a Marketing team can implement 
        to improve conversion rates and user engagement.
        
        Focus on changes that:
        1. Improve messaging clarity and impact
        2. Enhance call-to-action effectiveness  
        3. Optimize user journey and conversion funnel
        4. Address immediate usability issues
        5. Strengthen brand positioning
        
        Previous Analysis:
        {analysis_result}
        
        Website Context:
        - Title: {website_data['title']}
        - Current CTAs: {website_data['calls_to_action']}
        - Navigation: {website_data['navigation'][:5]}
        
        Provide recommendations in this format:
        
        🎯 PRIORITY RECOMMENDATIONS:
        
        1. [RECOMMENDATION TITLE]
           Problem: [What specific issue this addresses]
           Solution: [Specific actionable change]
           Impact: [Expected marketing/conversion benefit]
           Implementation: [How Marketing team can execute this]
        
        2. [Continue for 3-5 recommendations]
        
        💰 CONVERSION IMPACT ESTIMATE:
        [Brief assessment of potential conversion improvement]
        
        🚀 QUICK WINS:
        [1-2 changes that can be implemented immediately]
        
        Keep recommendations practical and focused on Marketing team capabilities.
        """
        
        try:
            response = self.client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=1200,
                temperature=0.4,
                messages=[
                    {"role": "user", "content": suggestion_prompt}
                ]
            )
            
            suggestions = response.content[0].text
            print("✅ Design suggestions generated!")
            return suggestions
            
        except Exception as e:
            print(f"❌ Error generating suggestions: {str(e)}")
            return f"Suggestion generation failed: {str(e)}"

# Initialize the Design Suggestion Agent
suggestion_agent = DesignSuggestionAgent(client)
print("💡 Design Suggestion Agent initialized!")

💡 Design Suggestion Agent initialized!


## 📈 Demo Value Proposition

**What the team just witnessed:**

✅ **Multi-Agent System**: 3 specialized AI agents working in coordination  
✅ **Real Website Analysis**: Live scraping and analysis of any website  
✅ **AI-Powered Insights**: Advanced analysis using Claude 3.5 Sonnet  
✅ **Marketing-Focused Output**: Actionable recommendations for Marketing teams  
✅ **30-Second Execution**: Rapid analysis that scales to any website  

**Immediate Business Value:**
- **Time Savings**: Replace hours of manual analysis with 30-second automated insights
- **Marketing Autonomy**: Enable Marketing team to analyze competitor sites independently  
- **Consistent Analysis**: Standardized evaluation framework across all websites
- **Actionable Output**: Specific recommendations, not just generic observations
- **Scalability**: Analyze hundreds of websites with the same effort as one

**Next Steps:**
1. Expand to local website rendering and modification
2. Add A/B testing and design option generation  
3. Build user-friendly interface for non-technical users
4. Integrate with existing Marketing tools and workflows

*This demo represents Day 1 of the 2-week development sprint. The foundation is solid and ready for rapid expansion!*

In [5]:
# 🚀 LIVE DEMO - Try It Now!

# Demo Configuration
DEMO_URLS = [
    "https://www.familysearch.org/en/campaign/temple-ord-ready",  # Simple, reliable test site
    "https://anthropic.com",  # AI company with modern design
    "https://github.com",  # Popular developer platform
    "https://stripe.com",  # Clean, conversion-focused design
]

print("🚀 AGENTIC WEBSITE ANALYSIS DEMO")
print("=" * 50)
print("Available demo URLs:")
for i, url in enumerate(DEMO_URLS, 1):
    print(f"  {i}. {url}")
print("\nTo run demo after all agents are initialized:")
print("  orchestrator.run_complete_analysis('URL_HERE')")
print("=" * 50)

# Quick demo function for easier testing
def quick_demo(url_index=1):
    """Run demo with one of the predefined URLs"""
    if url_index < 1 or url_index > len(DEMO_URLS):
        print(f"❌ Invalid URL index. Choose 1-{len(DEMO_URLS)}")
        return
    
    url = DEMO_URLS[url_index - 1]
    try:
        return orchestrator.run_complete_analysis(url)
    except NameError:
        print("❌ Error: Please run all agent initialization cells first!")
        print("   Required execution order:")
        print("   1. Setup cell (imports and dependencies)")
        print("   2. Web Acquisition Agent")
        print("   3. Analysis Agent") 
        print("   4. Design Suggestion Agent")
        print("   5. Demo Orchestrator")
        print("   6. Then you can run demos!")
        return None

print("💡 Quick demo function available: quick_demo(1) through quick_demo(4)")

🚀 AGENTIC WEBSITE ANALYSIS DEMO
Available demo URLs:
  1. https://www.familysearch.org/en/campaign/temple-ord-ready
  2. https://anthropic.com
  3. https://github.com
  4. https://stripe.com

To run demo after all agents are initialized:
  orchestrator.run_complete_analysis('URL_HERE')
💡 Quick demo function available: quick_demo(1) through quick_demo(4)
